<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Marc Spiegelman, Template from Kyle Mandli</td>
</table>


# Understand how to calculate compositional Jacobians 

For inclusion in models that calculate changes in composition by weight fraction, it is critical to calculate accurate jacobians of various thermodynamical quantities with respect to a change in composition which we usually calculate in weight percents.  However,  Coder always calculates variations with respect to changes in total moles of endmembers.  

Here we're going to try and sort some of this out starting with understanding changes of entropy in an ideal binary system 

In [ ]:
# load the standard goodies
import numpy as np
from scipy.optimize import fsolve, brentq
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline
plt.rcParams["figure.figsize"] = [8,6]
plt.rcParams['font.size'] = 16

### Load the ThermoCodegen object for the fo-fa reactive system

This will require that the python bindings for the py_fo_fa_binary module are in the python path

if you are using modules,  this can be accomplished by running
|
```bash
cd ../../reactions/fo_fa_binary
module load ./fo_fa_binary.module


In [ ]:
import py_fo_fa_binary as py_fo_fa
py_fo_fa.phase_info()

In [ ]:
rxn = py_fo_fa.fo_fa_binary()
rxn.report()

In [ ]:
Ol = py_fo_fa.Olivine()
Lq = py_fo_fa.Liquid()

utility functions 


In [ ]:
toCelsius = lambda T: T - 273.15
toKelvin = lambda T: T + 273.15

### make a contour plot of liquid entropy at fixed T,P

Make a meshgrid in (N1, N2) space and the simplex of mole fractions where $\mathbf{x}^T\mathbf{1}=1$

In [ ]:
eps = 1.e-3
n = np.linspace(eps, 1.2)
N1, N2 = np.meshgrid(n,n)
N_flat = np.array([ N1.flatten(),N2.flatten()]).T
    
x = np.linspace(eps,1.-eps,1001)
X = np.array([ x, 1.-x ]).T

Calculate entropy S in this space and along the simplex (Sx)

In [ ]:
T = toKelvin(1600.)
P = 1000.
S = np.array([ Lq.s(T,P,n) for n in N_flat])
S = S.reshape(N1.shape)
Sx = np.array( [ Lq.s(T,P,xx) for xx in X])

Calculate the gradient of entropy with respect to moles along the simplex 

In [ ]:
gradnS = -np.array([ Lq.d2gdndt(T,P, xx) for xx in X])

Choose a point $x_0$ on the simplex and calculate the gradient at that point for plotting

In [ ]:
x0 = np.array([0.1, 0.9])
gradnS0 = -np.array(Lq.d2gdndt(T,P,x0))

Plot entropy in the $N_1,N_2$ plane and its value and gradient along the simplex

In [ ]:
fig = plt.figure(figsize = (20,6))
ax = fig.add_subplot(1,3,1)
cp = ax.contourf(N1,N2,S,levels=11)
ax.plot([0., 1.], [1., 0.],'r--')
ax.plot(x0[0],x0[1],'ro')
ax.quiver(x0[0],x0[1],gradnS0[0],gradnS0[1],color='r', angles='xy', scale_units='xy', scale=2e3)
ax.quiver(x0[0],x0[1],1.,1.,color='k', angles='xy', scale_units='xy', scale=5)

ax.set_aspect('equal')
ax.set_xlabel('$n_1$')
ax.set_ylabel('$n_2$')
ax.set_title('$S(T,P,\\mathbf{n})$')

fig.colorbar(cp)
ax.grid()

ax = fig.add_subplot(1, 3, 2)
ax.plot(x, Sx)
ax.plot(x0[0],Lq.s(T,P,x0),'ro')
ax.set_xlabel('$x_1$')
ax.set_ylabel('s')
ax.grid()

ax = fig.add_subplot(1,3,3)
dx = np.array([eps, -eps])
dSx = gradnS.dot(dx)/np.linalg.norm(dx)
ax.plot(x, dSx)
#ax.plot(x, dsDx[:,0], label = 'Px Grad_n S')
ax.set_xlabel('$x_1$')
ax.set_ylabel('$\\frac{ds}{dx}$')
ax.set_title('$\\nabla_n S(T,P,\mathbf{x})\cdot\delta\mathbf{x}/||\delta\mathbf{x}||$')
ax.grid()

plt.show()

Check that $\delta S = \nabla_n s\cdot\delta\mathbf{x}$ is the proper expression for the directional derivative of $s$ along the simplex

In [ ]:
eps = 1.e-2
dx = np.array([eps, -eps])
points = [x0 - dx, x0, x0 + dx]

Sar = [ Lq.s(T,P,point) for point in points]
print(Sar)
print([Sar[1] + gradnS0.dot(-dx), Sar[1], Sar[1] + gradnS0.dot(dx)])
print(Sar[2]-Sar[1], Sar[0]-Sar[1])
print(gradnS0.dot(dx))

### Now let's consider the mapping from concentration vector $\mathbf{c}$ to mol fraction $\mathbf{x}$

We want to calculate two quantities
$$
    s(T,P,\mathbf{c}) = s(T,P,\mathbf{x}(\mathbf{c}))
$$

and
$$
    \nabla_c s ^T= \nabla_n s^T \frac{\partial\mathbf{x}}{\partial\mathbf{c}}
$$

such that 
$$\delta s = (\nabla_c s)^T\delta\mathbf{c} = (\nabla_n s)^T \delta\mathbf{x}$$

here
$$
    \frac{\partial\mathbf{x}}{\partial\mathbf{c}} = \frac{1}{\mathbf{c}^T(\mathbf{1/M})}\left( I -\mathbf{x}\mathbf{1}^T\right)\mathrm{diag}(\mathbf{1/M})
$$

is the Jacobian of the transformation 
$$
    \mathbf{x} = \frac{\mathrm{diag}(\mathbf{1/M})\mathbf{c}}{\mathbf{c}^T(\mathbf{1/M})} ~~~\left(\mathrm{and}~~\mathbf{c} = \frac{\mathrm{diag}(\mathbf{M})\mathbf{x}}{\mathbf{M}^T\mathbf{x}}\right)
$$
where $\mathbf{M}$ is a vector of molecular weights for each compositional component.

so we should be able to unroll all of this to return the vector
$$
    \nabla_c s^T   = \frac{1}{\mathbf{c}^T(\mathbf{1/M})}\left((\nabla_n s)^T\mathrm{diag}(\mathbf{1/M}) - (\nabla_n s)^T\mathbf{x}(\mathbf{1/M})^T\right)
$$


### let's try to code all of this up and check it out

In [ ]:
def gradnS(T,P,x,phase=Lq):
    return -np.array(phase.d2gdndt(T,P,x))

def gradcS(T,P,c,phase=Lq):
    K = len(c)
    iM = 1./np.array( [ phase.endmember_mw(k) for k in range(K)])
    sum = c.dot(iM)
    x = np.array(phase.c_to_x(c))
    gns = gradnS(T,P,x,phase=phase)
    gnc = ( gns - gns.dot(x))*iM/sum
    #gnc = gns.T.dot(dxdc(c))
    return gnc
    
def dxdc(c,phase=Lq):
    K = len(c)
    iM = 1./np.array( [ phase.endmember_mw(k) for k in range(K)])
    sum = c.dot(iM)
    x = np.array(phase.c_to_x(c))
    A = np.eye(K) - np.outer(x,np.ones(K))
    dxdc = 1./sum * A.dot(np.diag(iM))
    return dxdc

    

In [ ]:
c0 = np.array(Lq.x_to_c(x0))
Jx = dxdc(c0)
gns = gradnS(T,P,x0)
eps = 5.e-2
dc = np.array([eps, -eps])
dx = Jx.dot(dc)
gcs = gradcS(T,P,c0)
# and compare to tcg ds_dC
gcs_tcg = np.array(Lq.ds_dc(T,P,c0))


print('x0 = {}\nc0 = {}'.format(x0,c0))
print('dxdc = {}'.format(Jx))
print('dx = {}, dc = {}'.format(dx,dc))
print('grad_n s     ={}, dsx = {}'.format(gns, gns.dot(dx)))
print('grad_c s     ={}, dsc = {}'.format(gcs, gcs.dot(dc)))
print('grad_c s tcg ={}, dsc = {}'.format(gcs_tcg, gcs_tcg.dot(dc)))

### and check with some plots

In [ ]:
t = np.linspace(0.,1.)
c = np.array([t, 1.-t]).T
x = np.array([Lq.c_to_x(ci) for ci in c])

In [ ]:
print(gns.dot(dx),gcs.dot(dc))

In [ ]:
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(1,2,1)
ax.plot(c[:,0],x[:,0],'b',label='$x_1$')
ax.plot(c[:,0],c[:,0],'b--', label='$c_1$')
ax.plot(c[:,0],x[:,1],'r',label='$x_2$')
ax.plot(c[:,0],c[:,1],'r--',label='$c_2$')
ax.grid()
ax.set_xlabel('$c_1$')
ax.legend(loc='best')

ax = fig.add_subplot(1,2,2)
sx = np.array([ Lq.s(T,P,ci) for ci in c])
sc = np.array([Lq.s(T,P,xi) for xi in x])
ax.plot(t, sx, label='s(x)')
ax.plot(t, sc, label='s(c)')
s0 = Lq.s(T,P,x0)
ax.plot(x0[0],s0,'bo')
ax.plot(c0[0],s0,'ro')
ax.plot(x0[0]+dx[0],s0 + gns.dot(dx),'b+')
ax.plot(c0[0]+dc[0],s0 + gcs.dot(dc),'r+')


ax.set_xlabel('$x_1$ or $c_1$')
ax.set_ylabel('Entropy (J)')
ax.legend(loc='best')
ax.grid()

plt.show()

### Let's repeat this for chemical potentials $\mu_i^k$ for liquid endmembers 

The analysis should be the same except now $\nabla_n \mathbf{\mu}$ and $\nabla_c \mu$ are the matrices
$$
    \nabla_n \mu = \begin{bmatrix} \nabla_n \mu_1\\ \nabla_n\mu_2 \\ \vdots \\ \nabla_n\mu_k\end{bmatrix}
$$

and 

$$
    \nabla_c \mu  = \frac{1}{\mathbf{c}^T(\mathbf{1/M})}\nabla_n\mu\left(I - \mathbf{x}\mathbf{1}^T\right)\mathrm{diag}(\mathbf{1/M})
$$



In [ ]:
mu0 = np.array([ Lq.dgdn(T,P,n)[0] for n in N_flat])
mu1 = np.array([ Lq.dgdn(T,P,n)[1] for n in N_flat])
mu0 = mu0.reshape(N1.shape)
mu1 = mu1.reshape(N1.shape)
mu0x = np.array( [ Lq.mu(T,P,xx)[0] for xx in X])
mu1x = np.array( [ Lq.mu(T,P,xx)[1] for xx in X])

In [ ]:
x0 = np.array([0.1, 0.9])
gradnmu0 = np.array(Lq.d2gdn2(T,P,x0))[:2]
gradnmu1 = np.array(Lq.d2gdn2(T,P,x0))[1:]
print(gradnmu0, gradnmu1)
n0 = gradnmu0/np.linalg.norm(gradnmu0)
n1 = gradnmu1/np.linalg.norm(gradnmu1)

print(n0,n1)
eps = 1.e-3
dx = np.array([eps, -eps])
points = [x0 - dx, x0, x0 + dx]

mu0ar = [ Lq.dgdn(T,P,point)[0] for point in points]
mu1ar = [ Lq.dgdn(T,P,point)[1] for point in points]

print(mu0ar)
print([mu0ar[1] + gradnmu0.dot(-dx), mu0ar[1], mu0ar[1] + gradnmu0.dot(dx)])
print(mu1ar)
print([mu1ar[1] + gradnmu1.dot(-dx), mu1ar[1], mu1ar[1] + gradnmu1.dot(dx)])


In [ ]:
def gradnmu(T,P,x,phase=Lq):
    d2g = np.array(phase.d2gdn2(T,P,x))
    return np.array([d2g[:2], d2g[1:]])

def gradcmu(T,P,c,phase=Lq):
    K = len(c)
    iM = 1./np.array( [ phase.endmember_mw(k) for k in range(K)])
    sum = c.dot(iM)
    x = np.array(phase.c_to_x(c))
    gns = gradnmu(T,P,x,phase=phase)
    gnc = ( gns - gns.dot(x))*iM/sum
    #gnc = gns.T.dot(dxdc(c))
    return gnc

def gradcmu_tcg(T,P,c, phase=Lq):
    gc = np.array(phase.dmu_dc(T, P, c))
    return(gc)
    
    
    

In [ ]:
gnmu = gradnmu(T,P,x0)
gcmu = gradcmu(T,P,c0)
gcmu_tcg = gradcmu_tcg(T,P,c0)
print('grad_n mu     =\n{}'.format(gnmu))
print('grad_c mu     =\n{}'.format(gcmu))
print('grad_c_mu_tcg =\n{}'.format(gcmu_tcg))

### compare dmu_dc with dmu_dci wierdness from TCG

In [ ]:
eps = 1.e-2
dc = np.array([eps, -eps])
dx = dxdc(c0).dot(dc)
print('dc = {}'.format(dc))
print('dx = {}'.format(dx))
dng = gnmu.dot(dx)
dcg = gcmu.dot(dc)
print('d_n mu = {}'.format(dng))
print('d_c mu = {}'.format(dcg))


x1 = x0 + dx
c1 = c0 + dc
mux0 = Lq.mu(T,P,x0)
muc0 = Lq.mu(T,P,Lq.c_to_x(c0))

mux1 = np.array(Lq.mu(T,P,x1))
mux1p = mux0 + gradnmu(T,P,x0).dot(dx)
err = np.array(mux1 - mux1p)


print(mux1)
print(mux1p)
print(abs(err)/abs(mux1))

In [ ]:
dmu_dc_tcg = np.array(Lq.dmu_dc(T,P,c0))
dmu_dci_tcg = np.array(Lq.dmu_dci(T,P,c0,0))

In [ ]:
print(dmu_dc_tcg)
print(dmu_dci_tcg)
print(dmu_dc_tcg.dot(dc))
print(dmu_dci_tcg*dc[0])

In [ ]:
fig = plt.figure(figsize = (16,6))
ax = fig.add_subplot(1,2,1)
cp = ax.contourf(N1,N2,mu0,levels=11)
ax.plot([0., 1.], [1., 0.],'r--')
ax.plot(x0[0],x0[1],'ro')
ax.quiver(x0[0],x0[1],n0[0],n0[1],color='r', angles='xy', scale_units='xy', scale=2.5)
ax.quiver(x0[0],x0[1],1.,1.,color='k', angles='xy', scale_units='xy', scale=5)
ax.set_aspect('equal')
ax.set_xlabel('$n_1$')
ax.set_ylabel('$n_2$')
ax.set_title('$\\mu_{fo}^{Lq}(T,P,\\mathbf{n})$')

fig.colorbar(cp)
ax.grid()

ax = fig.add_subplot(1,2,2)
cp = ax.contourf(N1,N2,mu1,levels=11)
ax.plot([0., 1.], [1., 0.],'r--')
ax.plot(x0[0],x0[1],'ro')
ax.quiver(x0[0],x0[1],n1[0],n1[1],color='r', angles='xy', scale_units='xy', scale=2.5)
ax.quiver(x0[0],x0[1],1.,1.,color='k', angles='xy', scale_units='xy', scale=5)
ax.set_aspect('equal')
ax.set_xlabel('$n_1$')
ax.set_ylabel('$n_2$')
ax.set_title('$\\mu_{fa}^{Lq}(T,P,\\mathbf{n})$')

fig.colorbar(cp)
ax.grid()

plt.show()

In [ ]:
t = np.linspace(0.,1.)
c = np.array([t, 1.-t]).T
x = np.array([Lq.c_to_x(ci) for ci in c])

mux = np.array([ Lq.mu(T,P,ci) for ci in c])
muc = np.array([Lq.mu(T,P,xi) for xi in x])

In [ ]:
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(1,2,1)

ax.plot(t, mux[:,0], label='$\\mu_1(x)$')
ax.plot(t, muc[:,0], label='$\\mu_1(c)$')
ax.set_xlabel('$x_1$ or $c_1$')
ax.set_ylabel('$\mu$ (J)')
ax.legend(loc='best')

ax.plot(x0[0],mux0[0],'bo')
ax.plot(c0[0],muc0[0],'ro')
ax.plot(x0[0]+dx[0],mux0[0] + gnmu.dot(dx)[0],'b+')
ax.plot(c0[0]+dc[0],muc0[0] + gcmu.dot(dc)[0],'r+')

ax.grid()

ax = fig.add_subplot(1,2,2)
ax.plot(t, mux[:,1], label='$\\mu_2(x)$')
ax.plot(t, muc[:,1], label='$\\mu_2(c)$')
ax.plot(x0[0],mux0[1],'bo')
ax.plot(c0[0],muc0[1],'ro')
ax.plot(x0[0]+dx[0],mux0[1] + gnmu.dot(dx)[1],'b+')
ax.plot(c0[0]+dc[0],muc0[1] + gcmu.dot(dc)[1],'r+')

ax.set_xlabel('$x_1$ or $c_1$')
ax.set_ylabel('$\mu$ (J)')
ax.legend(loc='best')
ax.grid()
plt.show()